# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [27]:
# Dependencies and Setup

import pandas as pd
import requests
import hvplot.pandas

# Import API key
from api_keys import geoapify_key

In [28]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,bethel,41.3712,-73.4140,-2.66,63,100,0.89,US,1739629267
1,1,gokwe,-18.2048,28.9349,26.19,62,100,0.82,ZW,1739629268
2,2,kuraymah,18.5500,31.8500,28.51,8,2,6.07,SD,1739629270
3,3,jamestown,-15.9387,-5.7168,21.22,77,65,6.99,SH,1739629036
4,4,kodiak,57.7900,-152.4072,2.73,93,100,2.06,US,1739629272


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [30]:
city_data_df['Humidity'].describe()

count    592.000000
mean      70.625000
std       22.522211
min        5.000000
25%       59.000000
50%       75.000000
75%       88.000000
max      100.000000
Name: Humidity, dtype: float64

In [31]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [33]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df.loc[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
50,50,constantia,-34.0314,18.4183,23.77,71,0,3.58,ZA,1739629325
96,96,port alfred,-33.5906,26.8910,25.66,75,0,0.89,ZA,1739629377
112,112,al kharijah,25.4514,30.5464,23.85,10,0,2.83,EG,1739629396
159,159,east london,-33.0153,27.9116,24.16,64,0,4.03,ZA,1739629214
213,213,awjilah,29.1081,21.2869,21.12,18,0,4.25,LY,1739629511
237,237,necochea,-38.5473,-58.7368,25.04,47,0,2.97,AR,1739629538
277,277,bubaque,11.2833,-15.8333,23.79,53,0,3.08,GW,1739629584
375,375,dwarka,22.2394,68.9678,24.37,71,0,4.47,IN,1739629696
483,483,paradip garh,20.3167,86.6167,23.25,75,0,4.10,IN,1739629818
537,537,bechar,31.6167,-2.2167,24.18,12,0,3.09,DZ,1739629879


### Step 3: Create a new DataFrame called `hotel_df`.

In [35]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
50,constantia,ZA,-34.0314,18.4183,71,
96,port alfred,ZA,-33.5906,26.8910,75,
112,al kharijah,EG,25.4514,30.5464,10,
159,east london,ZA,-33.0153,27.9116,64,
213,awjilah,LY,29.1081,21.2869,18,
237,necochea,AR,-38.5473,-58.7368,47,
277,bubaque,GW,11.2833,-15.8333,53,
375,dwarka,IN,22.2394,68.9678,71,
483,paradip garh,IN,20.3167,86.6167,75,
537,bechar,DZ,31.6167,-2.2167,12,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [37]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
constantia - nearest hotel: Constantia Uitsig
port alfred - nearest hotel: No hotel found
al kharijah - nearest hotel: Kharga Hotel
east london - nearest hotel: No hotel found
awjilah - nearest hotel: No hotel found
necochea - nearest hotel: Cabañas Río Quequén
bubaque - nearest hotel: Chez Julio
dwarka - nearest hotel: The Dwarika Hotel
paradip garh - nearest hotel: Hotel Sea Pearl
bechar - nearest hotel: Hôtel Madania
pyu - nearest hotel: KMA


,City,Country,Lat,Lng,Humidity,Hotel Name
50,constantia,ZA,-34.0314,18.4183,71,Constantia Uitsig
96,port alfred,ZA,-33.5906,26.8910,75,No hotel found
112,al kharijah,EG,25.4514,30.5464,10,Kharga Hotel
159,east london,ZA,-33.0153,27.9116,64,No hotel found
213,awjilah,LY,29.1081,21.2869,18,No hotel found
237,necochea,AR,-38.5473,-58.7368,47,Cabañas Río Quequén
277,bubaque,GW,11.2833,-15.8333,53,Chez Julio
375,dwarka,IN,22.2394,68.9678,71,The Dwarika Hotel
483,paradip garh,IN,20.3167,86.6167,75,Hotel Sea Pearl
537,bechar,DZ,31.6167,-2.2167,12,Hôtel Madania


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [39]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)